In [230]:
import pandas as pd
import numpy as np
import pickle
from faker import Faker

# Initialize Faker
fake = Faker()

admission_types = [
    'URGENT', 'ELECTIVE', 'EW EMER.', 'DIRECT EMER.', 'EU OBSERVATION',
    'OBSERVATION ADMIT', 'DIRECT OBSERVATION',
    'AMBULATORY OBSERVATION', 'SURGICAL SAME DAY ADMISSION'
]
admission_locations = [
    'TRANSFER FROM HOSPITAL', 'TRANSFER FROM SKILLED NURSING FACILITY',
    'INTERNAL TRANSFER TO OR FROM PSYCH', 'PHYSICIAN REFERRAL',
    'EMERGENCY ROOM', 'PACU', 'PROCEDURE SITE',
    'WALK-IN/SELF REFERRAL', 'INFORMATION NOT AVAILABLE',
    'CLINIC REFERRAL'
]
insurance_types = [
    'Medicaid', 'Medicare', 'Other'
]
languages = [
    'ENGLISH', '?'
]
marital_statuses = [
    'SINGLE', 'MARRIED', 'DIVORCED', 'WIDOWED'
]
races = [
    'BLACK/CAPE VERDEAN', 'HISPANIC/LATINO - PUERTO RICAN', 'WHITE',
    'UNKNOWN', 'OTHER', 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - SALVADORAN', 'UNABLE TO OBTAIN',
    'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'HISPANIC/LATINO - CUBAN',
    'PATIENT DECLINED TO ANSWER', 'WHITE - BRAZILIAN',
    'HISPANIC OR LATINO'
]

drugs = [
    'Midodrine', 'Multivitamins W/minerals', 'Sodium Chloride',
    'Nephrocaps', 'Nicotine Polacrilex', 'OLANZapine', 'Ondansetron',
    'OxyCODONE (Immediate Release)'
]

diagnoses_types = [
    'Urinary tract infection, site not specified',
    'Acute respiratory failure',
    'Asthma, unspecified type, unspecified',
    'Solitary pulmonary nodule',
    'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
    'Dysphagia, oropharyngeal phase', 'Esophageal reflux', 'Hypoxemia',
    'Methicillin resistant pneumonia due to Staphylococcus aureus',
    'Attention deficit disorder with hyperactivity'
]

def generate_patients(n):
    patients = []
    for i in range(n):
        gender = np.random.choice(['M', 'F'])
        patients.append({
            'patient_id': i,
            'name': fake.name_male() if gender == "M" else fake.name_female(),
            'age': np.random.randint(1, 90),
            'gender': gender,
            'insurance': np.random.choice(insurance_types),
            'language': np.random.choice(languages),
            'maritalStatus': np.random.choice(marital_statuses),
            'race': np.random.choice(races),
        })
    return pd.DataFrame(patients)

def generate_staff(n):
    staff = []
    for i in range(n):
        staff.append({
            'staff_id': i,
            'name': fake.name(),
            'role': np.random.choice(['Physician', 'Nurse', 'Admin']),
            'shift_start': fake.time(),
            'shift_end': fake.time()
        })
    return pd.DataFrame(staff)

def generate_admissions(patients:pd.DataFrame, n):
    admissions = []
    for i in range(n):
        admissions.append({
            'adm_id': i,
            'patient_id': patients.sample(1).patient_id.values[0],
            'admit_time': fake.date_time(),
            'discharge_time': None,
            'type': np.random.choice(admission_types),
            'location': np.random.choice(admission_locations),
            
        })
    return pd.DataFrame(admissions)

def generate_omr(admissions):
    table = []
    for i in range(len(admissions)):
        omr = {
            'omr_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'weight': np.random.randint(60, 120),
            ##'height': np.random.randint(150, 220)/ 10.0, #not used in modeling
            'bp_systolic': np.random.randint(80, 220),
            'bp_diastolic': np.random.randint(60, 120),
        }
        table.append(omr)
    return pd.DataFrame(table)

def generate_prescriptions(admissions):
    prescriptions = []
    for i in range(len(admissions)):
        prescriptions.append({
            'prescription_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'drug': np.random.choice(drugs),
        })
    return pd.DataFrame(prescriptions)

def generate_diagnoses(admissions:pd.DataFrame):
    diag = []
    for i in range(len(admissions)):
        diag.append({
            'diagnosis_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'diagnosis': np.random.choice(diagnoses_types),
        })
    return pd.DataFrame(diag)

def generate_rooms(n):
    rooms = []
    beds = []
    bed_taken = 0
    room_id = 0
    bed_id = 0
    for i in range(n):
        if i % 2 == 0:#create new room
            rooms.append({
                'room_id': room_id,
                'capacity': 1,
            })
            beds.append({
                "bed_id": bed_id,
                "room": room_id,
                "adm_id": None
            })
            room_id += 1
        else:
            room = np.random.choice(rooms)#add bed to existing room
            room["capacity"] += 1
            beds.append({
                "bed_id": bed_id,
                "room": room["room_id"],
                "adm_id": None
            })
        bed_id += 1

    return pd.DataFrame(rooms), pd.DataFrame(beds)

def assign_beds(admissions, beds):
    for i in range(len(admissions)):
        adm = admissions.iloc[i].adm_id
        bed = beds[beds.adm_id.isna()].sample(1).iloc[0,0]  #choose random empty bed
        beds.loc[bed, "adm_id"] = adm.astype(int)
        print(bed)

In [231]:
staff = generate_staff(10)
patients = generate_patients(10)
admissions = generate_admissions(patients, 5)
admissions.discharge_time = pd.to_datetime(admissions.discharge_time)
omr = generate_omr(admissions)
prescriptions = generate_prescriptions(admissions)
diagnoses = generate_diagnoses(admissions)
rooms, beds = generate_rooms(20)
assign_beds(admissions, beds)

1
12
18
11
3


In [232]:
staff.head()

,staff_id,name,role,shift_start,shift_end
0,0,Tracy Miller,Admin,11:10:00,09:58:46
1,1,Michelle Colon,Physician,16:09:43,20:55:42
2,2,Brian Thompson,Nurse,11:31:32,19:31:20
3,3,Anne Lucas,Nurse,03:16:49,00:18:46
4,4,Michael Waters,Admin,13:08:55,16:45:08


In [233]:
patients.head()

,patient_id,name,age,gender,insurance,language,maritalStatus,race
0,0,Rachel Holmes,72,F,Other,?,SINGLE,UNKNOWN
1,1,Todd Osborne,84,M,Medicaid,ENGLISH,WIDOWED,UNABLE TO OBTAIN
2,2,Christopher Wong,74,M,Medicaid,?,MARRIED,OTHER
3,3,Virginia Price,6,F,Medicaid,ENGLISH,MARRIED,HISPANIC/LATINO - PUERTO RICAN
4,4,Jennifer Sims,51,F,Medicare,ENGLISH,MARRIED,OTHER


In [234]:
admissions

,adm_id,patient_id,admit_time,discharge_time,type,location
0,0,8,1981-12-03 17:18:25.156093,NaT,DIRECT OBSERVATION,WALK-IN/SELF REFERRAL
1,1,1,2001-04-02 04:54:53.959021,NaT,AMBULATORY OBSERVATION,PACU
2,2,7,2007-09-13 15:35:09.887802,NaT,URGENT,INFORMATION NOT AVAILABLE
3,3,7,2008-09-27 01:02:19.929131,NaT,EU OBSERVATION,CLINIC REFERRAL
4,4,6,1982-12-27 16:15:36.477452,NaT,ELECTIVE,PACU


In [235]:
omr.head()

,omr_id,adm_id,weight,bp_systolic,bp_diastolic
0,0,0,76,102,85
1,1,1,60,170,101
2,2,2,114,151,90
3,3,3,72,183,107
4,4,4,118,130,106


In [236]:
prescriptions.head()

,prescription_id,adm_id,drug
0,0,0,OxyCODONE (Immediate Release)
1,1,1,Ondansetron
2,2,2,OxyCODONE (Immediate Release)
3,3,3,OLANZapine
4,4,4,Sodium Chloride


In [237]:
diagnoses.head()

,diagnosis_id,adm_id,diagnosis
0,0,0,Streptococcus infection in conditions classifi...
1,1,1,Methicillin resistant pneumonia due to Staphyl...
2,2,2,Solitary pulmonary nodule
3,3,3,Esophageal reflux
4,4,4,"Asthma, unspecified type, unspecified"


In [238]:
rooms

,room_id,capacity
0,0,5
1,1,2
2,2,2
3,3,1
4,4,3
5,5,1
6,6,2
7,7,1
8,8,2
9,9,1


In [239]:
beds

,bed_id,room,adm_id
0,0,0,None
1,1,0,0
2,2,1,None
3,3,0,4
4,4,2,None
5,5,2,None
6,6,3,None
7,7,0,None
8,8,4,None
9,9,4,None


In [244]:
with open('app/data/staff.csv', 'w') as f:
    staff.to_csv(f, index=False)
with open('app/data/patients.csv', 'w') as f:
    patients.to_csv(f, index=False)
with open('app/data/admissions.csv', 'w') as f:
    admissions.to_csv(f, index=False)
with open('app/data/omr.csv', 'w') as f:
    omr.to_csv('app/data/omr.csv', index=False)
with open('app/data/prescriptions.csv', 'w') as f:
    prescriptions.to_csv("app/data/prescriptions.csv", index=False)
with open("app/data/diagnoses.csv", "w") as f:
    diagnoses.to_csv("app/data/diagnoses.csv", index=False)
with open("app/data/rooms.csv", "wb") as f:
    rooms.to_csv(f, index=False)
with open("app/data/beds.csv", "wb") as f:
    beds.to_csv(f, index=False)

In [241]:
with open("app/data/admission_types.pkl", "wb") as f:
    pickle.dump(admission_types, f)
with open("app/data/admission_locations.pkl", "wb") as f:
    pickle.dump(admission_locations, f)
with open("app/data/insurance_types.pkl", "wb") as f:
    pickle.dump(insurance_types, f)
with open("app/data/languages.pkl", "wb") as f:
    pickle.dump(languages, f)
with open("app/data/marital_statuses.pkl", "wb") as f:
    pickle.dump(marital_statuses, f)
with open("app/data/races.pkl", "wb") as f:
    pickle.dump(races, f)
with open("app/data/drugs.pkl", "wb") as f:
    pickle.dump(drugs, f)
with open("app/data/diagnoses_types.pkl", "wb") as f:
    pickle.dump(diagnoses_types, f)